## LTL*f* based trace alignment

In [1]:
import re
from typing import Match, cast
import copy
import numpy as np
import ast

from ltlf2dfa.parser.ltlf import LTLfParser
from dataclasses import dataclass
from typing import Dict, Match, Set, Tuple, cast

In [2]:
# path where load models and logs
root_folder = './RA/dataset/data/logs/' # to load dataset
root_folder_models = './RA/dataset/data/models/' # to load dataset

In [3]:
# test library usage
from ltlf2dfa.parser.ltlf import LTLfParser

parser = LTLfParser()
formula_str = "G(activity22_complete -> !(X(activity23_complete)))"
formula = parser(formula_str) 
dfa = formula.to_dfa(mona_dfa_out=True)

dfa

'DFA for formula with free variables: ACTIVITY22_COMPLETE ACTIVITY23_COMPLETE \nInitial state: 0\nAccepting states: 1 2 \nRejecting states: 0 3 \n\nAutomaton has 4 states and 7 BDD-nodes\nTransitions:\nState 0: XX -> state 1\nState 1: 0X -> state 1\nState 1: 1X -> state 2\nState 2: 00 -> state 1\nState 2: 01 -> state 3\nState 2: 10 -> state 2\nState 2: 11 -> state 3\nState 3: XX -> state 3\nA counter-example of least length (2) is:\nACTIVITY22_COMPLETE X 10\nACTIVITY23_COMPLETE X X1\n\nACTIVITY22_COMPLETE = {0}\nACTIVITY23_COMPLETE = {1}\n\nA satisfying example of least length (0) is:\nACTIVITY22_COMPLETE X \nACTIVITY23_COMPLETE X \n\nACTIVITY22_COMPLETE = {}\nACTIVITY23_COMPLETE = {}'

# Manage constraints automata

### Load constraints from TXT file

Below are loaded the synthetic version of constraints / models

In [4]:
'''
n_constr = "3_constraints"
file_n = "3constraints"
filename = root_folder_models + "synthetic" + "/" + n_constr + "/" + file_n + ".txt"
with open(filename) as f:
    lines3 = f.readlines()
    lines3 = [line.rstrip('\n') for line in lines10]
    print("******3 CONSTRAINTS*******")
    print(lines3)
'''

n_constr = "10_constraints"
file_n = "10constraints"
filename = root_folder_models + "synthetic" + "/" + n_constr + "/" + file_n + ".txt"
with open(filename) as f:
    lines10 = f.readlines()
    lines10 = [line.rstrip('\n') for line in lines10]
    print("******10 CONSTRAINTS*******")
    print(lines10)

n_constr = "15_constraints"
file_n = "15constraints"
filename = root_folder_models + "synthetic" + "/" + n_constr + "/" + file_n + ".txt"
with open(filename) as f:
    lines15 = f.readlines()
    lines15 = [line.rstrip('\n') for line in lines15]
    print("******15 CONSTRAINTS*******")
    print(lines15)

n_constr = "20_constraints"
file_n = "20constraints"
filename = root_folder_models + "synthetic" + "/" + n_constr + "/" + file_n + ".txt"
with open(filename) as f:
    lines20 = f.readlines()
    lines20 = [line.rstrip('\n') for line in lines20]
    print("******20 CONSTRAINTS*******")
    print(lines20)

******10 CONSTRAINTS*******
['((!activity10_complete) U activity9_complete) | G(!activity10_complete)', 'G(activity14_complete -> X(activity15_complete))', 'G(X(activity17_complete) -> activity16_complete) & (!activity17_complete)', 'F(activity19_complete) -> !(F(activity20_complete))', 'G(activity20_complete -> (!(F(activity21_complete))))', 'G(activity22_complete -> !(X(activity23_complete)))', 'F(activity5_complete) -> F(activity6_complete)', 'F(activity1_complete)', 'G(activity11_complete -> F(activity12_complete))', '!(F(activity2_complete & X(F(activity2_complete))))']
******15 CONSTRAINTS*******
['G(activity8_complete -> F(activity9_complete))', '((!activity10_complete) U activity9_complete) | G(!activity10_complete)', 'G(activity14_complete -> X(activity15_complete))', 'G(activity15_complete -> X(activity16_complete))', 'G(X(activity17_complete) -> activity16_complete) & (!activity17_complete)', 'G(X(activity18_complete) -> activity17_complete) & (!activity18_complete)', '!(F(a

In [5]:
lines10

['((!activity10_complete) U activity9_complete) | G(!activity10_complete)',
 'G(activity14_complete -> X(activity15_complete))',
 'G(X(activity17_complete) -> activity16_complete) & (!activity17_complete)',
 'F(activity19_complete) -> !(F(activity20_complete))',
 'G(activity20_complete -> (!(F(activity21_complete))))',
 'G(activity22_complete -> !(X(activity23_complete)))',
 'F(activity5_complete) -> F(activity6_complete)',
 'F(activity1_complete)',
 'G(activity11_complete -> F(activity12_complete))',
 '!(F(activity2_complete & X(F(activity2_complete))))']

### Use regular expressions to parse Deterministic Finite Automaton

In [6]:
class MonaOutputRegex():

    def __init__(self, output=dfa):
        self.output = output
        self.initial_state = "" 
        self.variable_names = "" 
        self.accepting_states = "" 
        self.rejecting_states = "" 
        self.transitions = "" 
    
    # generate all the combinations of 0 / 1 for 1 or 2 length
    def per(self, n):
        list_comb = []
        for i in range(1<<n):
            s=bin(i)[2:]
            s='0'*(n-len(s))+s
            list_comb.append(s)
        return list_comb
    
    # Create new state trasitions in case X is present as symbol
    def parseForXview(self, t):
        split_transition = t.split(" ")
        if "X" in split_transition[2]:  
            if len(split_transition[2]) == 2:
                if split_transition[2][0] == 'X' and split_transition[2][1] == 'X':
                    gen_perm = self.per(2) #generate couple of 0 and 1 for XX
                elif split_transition[2][0] == 'X' and split_transition[2][1] != 'X':
                    gen_perm = self.per(1)
                    print("First X ", gen_perm, split_transition[2][1])
                    gen_perm = [nxt + split_transition[2][1] for nxt in gen_perm]
                elif split_transition[2][0] != 'X' and split_transition[2][1] == 'X':
                    gen_perm = self.per(1)
                    print("Second X ",gen_perm, split_transition[2][0])
                    gen_perm = [split_transition[2][0] + nxt for nxt in gen_perm]
            elif len(split_transition[2][0]) == 1:
                gen_perm = self.per(1)
        else:
            gen_perm = [split_transition[2]]
        new_array_trans = []
        for elem in gen_perm:
            new_array_trans.append(split_transition[0] + " " + split_transition[1] + " " + elem + " " + split_transition[3] + " " + split_transition[4] + " " + split_transition[5])
            
        return new_array_trans
    
    def trans_func(self):
        raw_trans: Dict[int, Dict[int, Set[str]]] = {}
        lines = self.output.splitlines()
        print("LINES ", lines)
        trans_str = lines[7:] # takes the element with states transitions
        for t in trans_str:
            #if(t == "State 0: XX -> state 1"):
                # optimization additional state 0 returned by MONA
            #    continue
            match = cast(
                Match, re.search("State ([0-9]+): ([01X]+|) -> state ([0-9]+)", t)
            )
            if match is None:
                continue
            new_t = self.parseForXview(t)
            for t1 in new_t:
                print("T1 ---> ", t1)
                match = cast(
                    Match, re.search("State ([0-9]+): ([01X]+|) -> state ([0-9]+)", t1)
                )
                start_state = int(match.group(1))
                guard = match.group(2)
                end_state = int(match.group(3))
                raw_trans.setdefault(start_state, {}).setdefault(
                    end_state, set()
                ).add(guard)
        return raw_trans

    def create_output (self):
        name : Tuple[str, ...] = tuple(cast(Match, re.search("DFA for formula with free variables: (.*)", self.output),).group(1).split())
        init_state : int = int(cast(Match, re.search("Initial state: (.*)\n", self.output)).group(1))
        acpt_states : Set[int] = set(map(int,cast(Match, re.search("Accepting states: (.*)\n", self.output)).group(1).split(),))
        rej_states : Set[int] = set(map(int,cast(Match, re.search("Rejecting states: (.*)\n", self.output)).group(1).split(),))
        #nb_states : int = int(cast(Match,re.search(r"Automaton has ([0-9]+) state(\(?s\)?)? and .* BDD-node(\(?s\)?)?",self.output,),).group(1))
        raw_trans : Dict[int, Dict[int, Set[str]]] = self.trans_func()
            
        self.initial_state = init_state
        self.variable_names = name
        self.accepting_states = acpt_states
        self.rejecting_states = rej_states
        self.transitions = raw_trans
        
        return self
        # return MonaOutput(nb_states, name, init_state, acpt_states, rej_states, raw_trans)
        #return MonaOutput(name, init_state, acpt_states, rej_states, raw_trans)

### Construct Constraints automata A+ parsing the DFA with Regular Expressions

In [7]:
index = 1
index_states = 0

automata_constraints = {}

# iterate over the constraints saved, parse it and translate into a DFA
#for f in lines10:
for f in lines15:
    #index_states = 0
    build_automaton = {}
    build_automaton["formula"] = f

    all_states_constr = []
    final_states_constr = []
    init_states_constr = []
    rejecting_states_constr = []
    trans_base = []
    trans_negated = []
    trans_positive = []

    ## Parser & dfa 
    formula = parser(f)       # returns an LTLfFormula
    dfa = formula.to_dfa(mona_dfa_out=True)
    print("DFA **** ", dfa)
    #mona_output = MonaOutputRegex(output=dfa).create_output() 
    mona_output = MonaOutputRegex(output=dfa).create_output() 
    ## define the initial state 
    init_states_constr = 's'+str(index_states)
    
    ## define all the states
    states = set.union(mona_output.rejecting_states, mona_output.accepting_states)
    states = list(states)
    for elem in states:
        all_states_constr.append('s'+str(elem+index_states))
    
    ## define the accepting state 
    for elem in list(mona_output.accepting_states):
        final_states_constr.append('s'+str(elem+index_states))
    
    # define the alphabet 
    alphabet = mona_output.variable_names
    
    
    # define the transitions
    for key, elem in mona_output.transitions.items():
        s_start = key + index_states
        for k, e in elem.items():
            
            list_transition = list(e)
            for next_val in list_transition:
                s_end = k + index_states
                sum = 0
                for char in next_val:
                    if(char!='X'):
                        sum += int(char)
                # work with singleton
                if(sum > 1):
                    continue
                elif(sum == 0):
                    idx0 = [i for i in range(len(next_val)) if next_val[i] in '0']
                    if (len(idx0) == 0):
                        # there are X, add in transition positive
                        idx0 = [i for i in range(len(next_val)) if next_val[i] in 'X']
                        res_list = [alphabet[i] for i in idx0]
                        trans_positive.append([s_start,s_end,res_list])

                    else:
                        # there are all 0, add in transition negative
                        res_list = [alphabet[i] for i in idx0]
                        trans_negated.append([s_start,s_end,res_list])

                elif(sum == 1):
                    # there is only a single 1
                    idx1 = next_val.find('1')
                    trans_base.append("s"+str(s_start)+" "+alphabet[idx1]+" s"+str(s_end))
                             

    print("BASE ", trans_base)
    print("POSITIVE ", trans_positive)
    print("NEGATIVE ", trans_negated)
    build_automaton["init_state"] = init_states_constr
    build_automaton["final_states"] = final_states_constr
    build_automaton["all_states"] = all_states_constr
    build_automaton["symbols_constr"] = alphabet
    build_automaton["transitions"] = trans_base
    build_automaton["negated_transitions"] = trans_negated
    build_automaton["positive_transitions"] = trans_positive
    
    # defined an automaton for each constraints/DFA
    automata_constraints["a"+str(index)] = build_automaton

    index += 1
    index_states += len(all_states_constr)

DFA ****  DFA for formula with free variables: ACTIVITY8_COMPLETE ACTIVITY9_COMPLETE 
Initial state: 0
Accepting states: 1 
Rejecting states: 0 2 

Automaton has 3 states and 4 BDD-nodes
Transitions:
State 0: XX -> state 1
State 1: 0X -> state 1
State 1: 10 -> state 2
State 1: 11 -> state 1
State 2: X0 -> state 2
State 2: X1 -> state 1
A counter-example of least length (1) is:
ACTIVITY8_COMPLETE X 1
ACTIVITY9_COMPLETE X 0

ACTIVITY8_COMPLETE = {0}
ACTIVITY9_COMPLETE = {}

A satisfying example of least length (0) is:
ACTIVITY8_COMPLETE X 
ACTIVITY9_COMPLETE X 

ACTIVITY8_COMPLETE = {}
ACTIVITY9_COMPLETE = {}
LINES  ['DFA for formula with free variables: ACTIVITY8_COMPLETE ACTIVITY9_COMPLETE ', 'Initial state: 0', 'Accepting states: 1 ', 'Rejecting states: 0 2 ', '', 'Automaton has 3 states and 4 BDD-nodes', 'Transitions:', 'State 0: XX -> state 1', 'State 1: 0X -> state 1', 'State 1: 10 -> state 2', 'State 1: 11 -> state 1', 'State 2: X0 -> state 2', 'State 2: X1 -> state 1', 'A counter

DFA ****  DFA for formula with free variables: ACTIVITY17_COMPLETE ACTIVITY16_COMPLETE 
Initial state: 0
Accepting states: 1 2 
Rejecting states: 0 3 

Automaton has 4 states and 5 BDD-nodes
Transitions:
State 0: XX -> state 1
State 1: 00 -> state 1
State 1: 01 -> state 2
State 1: 1X -> state 3
State 2: X0 -> state 1
State 2: X1 -> state 2
State 3: XX -> state 3
A counter-example of least length (1) is:
ACTIVITY17_COMPLETE X 1
ACTIVITY16_COMPLETE X X

ACTIVITY17_COMPLETE = {0}
ACTIVITY16_COMPLETE = {}

A satisfying example of least length (0) is:
ACTIVITY17_COMPLETE X 
ACTIVITY16_COMPLETE X 

ACTIVITY17_COMPLETE = {}
ACTIVITY16_COMPLETE = {}
LINES  ['DFA for formula with free variables: ACTIVITY17_COMPLETE ACTIVITY16_COMPLETE ', 'Initial state: 0', 'Accepting states: 1 2 ', 'Rejecting states: 0 3 ', '', 'Automaton has 4 states and 5 BDD-nodes', 'Transitions:', 'State 0: XX -> state 1', 'State 1: 00 -> state 1', 'State 1: 01 -> state 2', 'State 1: 1X -> state 3', 'State 2: X0 -> state 1

DFA ****  DFA for formula with free variables: ACTIVITY11_COMPLETE ACTIVITY12_COMPLETE 
Initial state: 0
Accepting states: 1 
Rejecting states: 0 2 

Automaton has 3 states and 4 BDD-nodes
Transitions:
State 0: XX -> state 1
State 1: 0X -> state 1
State 1: 10 -> state 2
State 1: 11 -> state 1
State 2: X0 -> state 2
State 2: X1 -> state 1
A counter-example of least length (1) is:
ACTIVITY11_COMPLETE X 1
ACTIVITY12_COMPLETE X 0

ACTIVITY11_COMPLETE = {0}
ACTIVITY12_COMPLETE = {}

A satisfying example of least length (0) is:
ACTIVITY11_COMPLETE X 
ACTIVITY12_COMPLETE X 

ACTIVITY11_COMPLETE = {}
ACTIVITY12_COMPLETE = {}
LINES  ['DFA for formula with free variables: ACTIVITY11_COMPLETE ACTIVITY12_COMPLETE ', 'Initial state: 0', 'Accepting states: 1 ', 'Rejecting states: 0 2 ', '', 'Automaton has 3 states and 4 BDD-nodes', 'Transitions:', 'State 0: XX -> state 1', 'State 1: 0X -> state 1', 'State 1: 10 -> state 2', 'State 1: 11 -> state 1', 'State 2: X0 -> state 2', 'State 2: X1 -> state 1'

In [8]:
automata_constraints

{'a1': {'formula': 'G(activity8_complete -> F(activity9_complete))',
  'init_state': 's0',
  'final_states': ['s1'],
  'all_states': ['s0', 's1', 's2'],
  'symbols_constr': ('ACTIVITY8_COMPLETE', 'ACTIVITY9_COMPLETE'),
  'transitions': ['s0 ACTIVITY9_COMPLETE s1',
   's0 ACTIVITY8_COMPLETE s1',
   's1 ACTIVITY9_COMPLETE s1',
   's1 ACTIVITY8_COMPLETE s2',
   's2 ACTIVITY8_COMPLETE s2',
   's2 ACTIVITY9_COMPLETE s1'],
  'negated_transitions': [[0, 1, ['ACTIVITY8_COMPLETE', 'ACTIVITY9_COMPLETE']],
   [1, 1, ['ACTIVITY8_COMPLETE', 'ACTIVITY9_COMPLETE']],
   [2, 2, ['ACTIVITY8_COMPLETE', 'ACTIVITY9_COMPLETE']]],
  'positive_transitions': []},
 'a2': {'formula': '((!activity10_complete) U activity9_complete) | G(!activity10_complete)',
  'init_state': 's3',
  'final_states': ['s4', 's5'],
  'all_states': ['s3', 's4', 's5', 's6'],
  'symbols_constr': ('ACTIVITY10_COMPLETE', 'ACTIVITY9_COMPLETE'),
  'transitions': ['s3 ACTIVITY9_COMPLETE s4',
   's3 ACTIVITY10_COMPLETE s4',
   's4 ACTIVITY9_C

# Manage trace automata

### Load 10 / 15 / 20 constraints syntetic

Below are loaded the synthetic version of logs

In [9]:
log_paths_syn = {}
n_constraints = ["10"] 
#n_constraints = ["15"] 
#n_constraints = ["20"]
choice = "SYN"
constraints_inverted = ["3", "4", "6"] # even 1 and 2 ## for 10 constraints
#constraints_inverted = ["4", "6", "8"] # even 1 and 3 ## for 15 constraints
#constraints_inverted = ["4", "6", "8"] # even 1 and 2 ## for 20 constraints
len_traces = ["1-50"] #,"51-100", "101-150", "151-200"] ## for 10, 15, 20 constraints

for i in range(len(n_constraints)):
    for j in range(len(constraints_inverted)):
        for k in range(len(len_traces)):
            #DEL key = choice+"/"+n_constraints[i]+"_"+constraints_inverted[j]+"/"+len_traces[k]
            key = choice+"_"+n_constraints[i]+"_"+constraints_inverted[j]+"_"+len_traces[k]
            log_paths_syn[key] = root_folder +"synthetic"+"/"+n_constraints[i]+"_constraints/"+constraints_inverted[j]+"_constraints_inverted/log-"+len_traces[k]+".xes"
            #log_paths_syn[key] = root_folder +"synthetic"+"/"+n_constraints[i]+"_constraints/"+constraints_inverted[j]+"_constraints_inverted/TEST_log-"+len_traces[k]+".xes"
        
        # manage the 15 constraints that contains 2 files (201-205) for constraints_inverted 3 and 6
        if n_constraints[i] == '15' and constraints_inverted[j] in ('3','6'):
            #DEL key = choice+"/"+n_constraints[i]+"_"+constraints_inverted[j]+"/"+"201-250"
            key = choice+"_"+n_constraints[i]+"_"+constraints_inverted[j]+"_"+"201-250"
            log_paths_syn[key] = root_folder +"synthetic"+"/"+n_constraints[i]+"_constraints/"+constraints_inverted[j]+"_constraints_inverted/log-"+"201-250"+".xes"
            

In [10]:
log_paths_syn
#'SYN_10_1_1-50' , SYN_10_1_51-100', ... --> used to identify the trace

{'SYN_10_3_1-50': './RA/dataset/data/logs/synthetic/10_constraints/3_constraints_inverted/log-1-50.xes',
 'SYN_10_4_1-50': './RA/dataset/data/logs/synthetic/10_constraints/4_constraints_inverted/log-1-50.xes',
 'SYN_10_6_1-50': './RA/dataset/data/logs/synthetic/10_constraints/6_constraints_inverted/log-1-50.xes'}

### Convert xes files to string traces

In [11]:
def parseXesLogTraces (log_path):
    '''
        event_bool:     True when we have to read an event from the file
        trace:          List of events as integers
        state_event:    List of "kind" of events
        traceAct:       List of events as strings
        traceString:    String of events
        log:            List of all traces (as strings)
    '''

    event_bool = False
    trace = []
    state_event = []
    traceAct = []
    traceString = ""
    log = []

    f = open(log_path)
    f1 = f.readlines()
    
    for x in f1:
        # if there is an event, we activate the event_bool
        if (x.__contains__("<event>")):
            event_bool = True

        # If event_bool is True and the name of the event is present,
        # we extract the event name and append it to "trace"
        if (event_bool and x.__contains__('<string key="concept:name"')):
            val = x.split('value="activity ',1)[1]
            val = val.split('"')[0]  
            trace.append(val)
        
        if (event_bool and x.__contains__('string key="lifecycle:transition"')):
            status = x.split('value="',1)[1]
            status = status.split('"/>',1)[0]  
            state_event.append(status.upper())
            event_bool = False
        
        # When the trace is terminated, we can convert the numbers in strings
        if (x.__contains__("</trace>")):
            # trace variable contains all the activity nums of the current trace [10, 15, 11, ....]
            for event,i in zip(trace,range(0,len(trace))):
                # construct the event
                traceAct.append("ACTIVITY"+str(event)+"_"+state_event[i])
                
            # create a unique string trace
            traceString = ";".join(traceAct)
            # traceString = KGUNOYIVCTKBOCFFFTWDMAJWLNOI, traceString = LHVFBLCNOMRAUYHMEYCFPU, ecc
            log.append(traceString)
            
            # Reset variables
            trace = []
            state_event = []
            traceAct = []
            traceString = ""
    
    return log

### TRACE automata A+ construction

In [12]:
path_traces_log = copy.deepcopy(log_paths_syn)
automata_const_log = {}

for k, log_path in log_paths_syn.items():
    traces_log = parseXesLogTraces(log_path) # list of traces (string ";", each element of the array is a sequence of events of the nth trace)
    all_traces = {}
    variabili = {}

    idx_trace = 1
    
    for t in traces_log:
        trace = {}
        variabile = copy.deepcopy(automata_constraints)
        
        #symbols_trace = list(set(t)) # define the set of unique symbol for each trace
        
        ## Build trace automaton ###################################
        rho_trace_basic = []
        Q_trace = []
        next_act = t.split(";")
        symbols_trace = np.unique(next_act)
        for i, act in zip(range(len(t)),next_act):
            Q_trace.append('t'+str(i))
            rho_trace_basic.append('t'+str(i)+" "+ act +" "+ 't'+str(i+1))

        Q_trace.append('t'+str(len(Q_trace)))
        init_state_trace = Q_trace[0]
        final_state_trace = Q_trace[-1]
        
        trace["symbols_trace"] = symbols_trace
        trace["init_state_trace"] = init_state_trace
        trace["final_state_trace"] = final_state_trace
        trace["Q_trace"] = Q_trace
        trace["rho_trace_basic"] = rho_trace_basic
        all_traces[idx_trace] = trace

        ## Now we add the transitions for the negated symbols ######

        for ID,automaton in automata_constraints.items():
            symb_constr = set(variabile[ID]["symbols_constr"])
            symb_trace = set(symbols_trace)
            all_symbs = list(set.union(symb_constr, symb_trace))

            trans = copy.deepcopy(automaton["transitions"])
            for t_neg in variabile[ID]["negated_transitions"]:
                symbs = copy.deepcopy(all_symbs)
                for t in t_neg[2]:
                    symbs.remove(t)
                for elem in symbs:
                    if ("s"+str(t_neg[0])+" "+elem+" s"+str(t_neg[1]) not in trans):
                        trans.append("s"+str(t_neg[0])+" "+elem+" s"+str(t_neg[1]))

            for t_pos in variabile[ID]["positive_transitions"]:
                symbs = copy.deepcopy(all_symbs)
                for elem in symbs:
                    if ("s"+str(t_pos[0])+" "+elem+" s"+str(t_pos[1]) not in trans):
                        trans.append("s"+str(t_pos[0])+" "+elem+" s"+str(t_pos[1]))

            variabile[ID]["transitions"] = trans

        variabili[idx_trace] = variabile

        idx_trace += 1

    automata_const_log[k] = variabili

    path_traces_log[k] = all_traces
    


In [13]:
# DOMAIN Definition

domain_name = "domain_trace_alignment"

pddl_domain_initial = "(define (domain domain_trace_alignment) "\
                "(:requirements :typing :disjunctive-preconditions :conditional-effects :universal-preconditions :action-costs) "\
                "(:types trace_state automaton_state - state activity) "

pddl_domain_predicates = "(:predicates "\
                            "(trace ?t1 - trace_state ?e - activity ?t2 - trace_state) "\
                            "(cur_state ?s - state) "\
                            "(automaton ?s1 - automaton_state ?e - activity ?s2 - automaton_state) "\
                            "(final_state ?s - state) "\
                            ") "\
                            "(:functions total-cost) "


pddl_domain_actions =  "(:action sync "\
                        ":parameters (?t1 - trace_state ?e - activity ?t2 - trace_state) "\
                        ":precondition (and (cur_state ?t1)(trace ?t1 ?e ?t2)) "\
                        ":effect (and "\
                        "(not (cur_state ?t1)) "\
                        "(cur_state ?t2) "\
                        "(forall (?s1 ?s2 - automaton_state) "\
                        "(when (and (cur_state ?s1) (automaton ?s1 ?e ?s2)) "\
                        "(and (not (cur_state ?s1))(cur_state ?s2)))))) "\
                        "(:action add "\
                        ":parameters (?e - activity) "\
                        ":effect (and "\
                        "(increase (total-cost) 1) "\
                        "(forall (?s1 ?s2 - automaton_state) "\
                        "(when (and (cur_state ?s1) (automaton ?s1 ?e ?s2)) "\
                        "(and (not (cur_state ?s1))(cur_state ?s2)))))) "\
                        "(:action del "\
                        ":parameters (?t1 - trace_state ?e - activity ?t2 - trace_state) "\
                        ":precondition (and (cur_state ?t1)(trace ?t1 ?e ?t2)) "\
                        ":effect(and "\
                        "(increase (total-cost) 1) "\
                        "(not (cur_state ?t1)) (cur_state ?t2))))"

pddl_domain = pddl_domain_initial + pddl_domain_predicates + pddl_domain_actions

# SAVE the pddl domain file

file1 = open("RA/PDDL/"+"synthetic"+"/"+domain_name+".pddl", "w")
file1.write(pddl_domain)
file1.close()


## PDDL

In [14]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

for key,all_traces in path_traces_log.items():
    for id_trace, trace in all_traces.items():
        automata_constraints = copy.deepcopy(automata_const_log[key][id_trace])
        all_states_constr = []
        for ID,a in automata_constraints.items():
            for elem in a['all_states']:
                all_states_constr.append(elem)

        symbols_tot = []
        for ID,a in automata_constraints.items():
            for s in a["symbols_constr"]:
                symbols_tot.append(s)
        symbols_tot = list(set(symbols_tot).union(set(trace["symbols_trace"])))
        
        
        # PROBLEM Definition
        problem_name = "problem_"+key+"_trace"+str(id_trace)
        pddl_problem_initial = "(define (problem "+problem_name+") (:domain domain_trace_alignment) "
        
        # Define the part related to the log trace
        pddl_problem_init = "(:init (= (total-cost) 0) (cur_state "+trace["init_state_trace"]+") "
        for t in trace["rho_trace_basic"]:
            pddl_problem_init += "(trace "+t+") "
        pddl_problem_init += "(final_state "+trace["final_state_trace"]+") "

        # Define the part related to the constraints
        for ID, a in automata_constraints.items():
            #pddl_problem_init += "(cur_state "+a["init_state"]+") " 
            all_states_constr.remove(a["init_state"].strip())
            init_recalc = int(a["init_state"][1:])+1 # redefine the init state of the constraint automaton
            pddl_problem_init += "(cur_state "+'s'+str(init_recalc)+") " 
            for t in a["transitions"]:
                array = t.split(" ")
                # Optimization --> delete transitions from s0 and transition from sX --> sX
                if(array[0] == array[2] or array[0] == a["init_state"]):
                    continue
                pddl_problem_init += "(automaton "+t+") "
                

            for i in range(len(a["final_states"])):
                pddl_problem_init += "(final_state "+a["final_states"][i]+") "
        pddl_problem_init += ") "
        
        # Define the objects, in all_states_constr has been removed the initial states defined by MONA
        pddl_problem_objects = "(:objects "
        
        for q in trace["Q_trace"]:
            pddl_problem_objects += q+" "
        pddl_problem_objects += "- trace_state "
        
        for q in all_states_constr:
            pddl_problem_objects += q+" "
        pddl_problem_objects += "- automaton_state "
        
        for s in symbols_tot:
            pddl_problem_objects += s+" "
        
        pddl_problem_objects += "- activity"
        pddl_problem_objects += ") "


        pddl_problem_goal = "(:goal (forall (?s - state) "\
                            "(imply (cur_state ?s) (final_state ?s)))) "
        pddl_problem_metric = "(:metric minimize (total-cost)))"
        pddl_problem = pddl_problem_initial + pddl_problem_objects + pddl_problem_init + pddl_problem_goal + pddl_problem_metric

        # SAVE the pddl problem files

        file2 = open("RA/PDDL/"+"synthetic"+"/"+problem_name+".pddl", "w")
        file2.write(pddl_problem)
        file2.close()

        id_trace += 1


now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)


Current Time = 10:24:50
Current Time = 10:24:51
